In [6]:
import pandas as pd
import yaml
import requests
import json

In [7]:
secrets_location = "C:\\Users\\lowsona\\Github\\Supporting Files\\secrets.yml"
secrets = open(secrets_location)
secrets = yaml.load(secrets, Loader=yaml.FullLoader)['SharePoint_LGBF']
SP_clientsecret = secrets['client_secret']
SP_tenantid = secrets['tenant_id']
SP_tenant = "stirlingcounciluk"
SP_clientid = secrets['client_id'] + '@' + SP_tenantid

In [8]:
# Establish data to be passed as part of POST request to obtain session token from SharePoint
data = {
    'grant_type': 'client_credentials',
    'resource': "00000003-0000-0ff1-ce00-000000000000/" + SP_tenant + ".sharepoint.com@" + SP_tenantid,
    'client_id': SP_clientid,
    'client_secret': SP_clientsecret,
}

# Establish headers to be passed as part of POST request to obtain session token from SharePoint
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Construct url variable for token request
url = "https://accounts.accesscontrol.windows.net/" + \
    SP_tenantid + "/tokens/OAuth/2"

# Send POST request to obtain session token and then load the response into json_data
r = requests.post(url, data=data, headers=headers)
json_data = json.loads(r.text)

In [9]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept': 'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + \
    ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Alternate Line Types')/items"

Items = []

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.
def getItems(url):
    Itemsrequest = requests.get(url, headers=headers)
    Itemsjson = Itemsrequest.json()
    Item = pd.json_normalize(Itemsjson['d']['results'])
    Items.append(Item)
    if '__next' in Itemsjson['d']:
        newurl = Itemsjson['d']['__next']
        getItems(newurl)


getItems(url)

alternateLines = pd.DataFrame()
alternateLines = pd.concat(Items)

alternateLines

,FileSystemObjectType,Id,ServerRedirectedEmbedUri,ServerRedirectedEmbedUrl,ID,ContentTypeId,Title,Modified,Created,AuthorId,...,FieldValuesAsText.__deferred.uri,FieldValuesForEdit.__deferred.uri,File.__deferred.uri,Folder.__deferred.uri,LikedByInformation.__deferred.uri,ParentList.__deferred.uri,Properties.__deferred.uri,Versions.__deferred.uri,IndicatorId.__metadata.type,IndicatorId.results
0,0,1,None,,1,0x0100A3C8811D84B3D24FA26C040BA00BBD9500892711...,None,2023-04-28T11:20:49Z,2021-03-30T09:40:37Z,10,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,Collection(Edm.Int32),"[79, 80, 81, 82, 85, 86, 87, 88, 89, 90]"


In [10]:
# Drop system columns not needed in dataframe
alternateLines = alternateLines.drop(columns=['FileSystemObjectType', 'Id', 'ServerRedirectedEmbedUri', 'ServerRedirectedEmbedUrl', 'GetDlpPolicyTip.__deferred.uri', 'FieldValuesAsHtml.__deferred.uri', 'FieldValuesAsText.__deferred.uri', 'FieldValuesForEdit.__deferred.uri', 'File.__deferred.uri', 'Folder.__deferred.uri', 'LikedByInformation.__deferred.uri', 'ParentList.__deferred.uri', 'Properties.__deferred.uri',
                                                 'Versions.__deferred.uri', 'Attachments', 'GUID', '__metadata.id', '__metadata.uri', '__metadata.etag', '__metadata.type', 'FirstUniqueAncestorSecurableObject.__deferred.uri', 'RoleAssignments.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'ContentType.__deferred.uri', 'ID', 'Modified', 'Created', 'AuthorId', 'EditorId', 'OData__UIVersionString', 'ContentTypeId', 'ComplianceAssetId','IndicatorId.__metadata.type','Title'])

# Rename columns appropriately
alternateLines = alternateLines.rename(columns={'PeriodsAffected_x0028_semicolond': 'Period','AlternateLineType_x0028_IfApplic' : 'LineType','IndicatorId.results': 'IndicatorID'})
alternateLines


,Period,LineType,OData__ColorTag,IndicatorID
0,2019-20;2020-21;2021-22,Dashed,None,"[79, 80, 81, 82, 85, 86, 87, 88, 89, 90]"


In [11]:
alternateLines = alternateLines.assign(Period=alternateLines.Period.str.split(';'))
alternateLines = alternateLines.explode('Period')
alternateLines = alternateLines.explode('IndicatorID')
alternateLines

,Period,LineType,OData__ColorTag,IndicatorID
0,2019-20,Dashed,None,79
0,2019-20,Dashed,None,80
0,2019-20,Dashed,None,81
0,2019-20,Dashed,None,82
0,2019-20,Dashed,None,85
0,2019-20,Dashed,None,86
0,2019-20,Dashed,None,87
0,2019-20,Dashed,None,88
0,2019-20,Dashed,None,89
0,2019-20,Dashed,None,90


In [12]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept': 'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + \
    ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Indicator%20Information')/items"

Items = []

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.


def getItems(url):
    Itemsrequest = requests.get(url, headers=headers)
    Itemsjson = Itemsrequest.json()
    Item = pd.json_normalize(Itemsjson['d']['results'])
    Items.append(Item)
    if '__next' in Itemsjson['d']:
        newurl = Itemsjson['d']['__next']
        getItems(newurl)


getItems(url)

indicatorInformation = pd.DataFrame()
for i in Items:
    indicatorInformation = indicatorInformation.append(i)

indicatorInformation = indicatorInformation[['Id','Code_x0028_SortableA_x002d_Z_x00']]
indicatorInformation

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
alternateLines = alternateLines.merge(indicatorInformation, left_on = ['IndicatorID'], right_on=['Id'])
alternateLines = alternateLines[['Code_x0028_SortableA_x002d_Z_x00','Period','LineType']]
alternateLines = alternateLines.rename(columns={'Code_x0028_SortableA_x002d_Z_x00': 'Code'})
alternateLines['Key_CodePeriod'] = alternateLines['Code'] + alternateLines['Period']
alternateLines

,Code,Period,LineType,Key_CodePeriod
0,CHN 04,2019-20,Dashed,CHN 042019-20
1,CHN 04,2020-21,Dashed,CHN 042020-21
2,CHN 04,2021-22,Dashed,CHN 042021-22
3,CHN 05,2019-20,Dashed,CHN 052019-20
4,CHN 05,2020-21,Dashed,CHN 052020-21
5,CHN 05,2021-22,Dashed,CHN 052021-22
6,CHN 06,2019-20,Dashed,CHN 062019-20
7,CHN 06,2020-21,Dashed,CHN 062020-21
8,CHN 06,2021-22,Dashed,CHN 062021-22
9,CHN 07,2019-20,Dashed,CHN 072019-20


In [ ]:
alternateLines.to_csv("Data Files//Alternative Lines.csv", index=False, encoding='utf-8-sig')